In [1]:
import numpy as np
import pandas as pd
import os, sys, datetime
######### Pysaprk
from pyspark import SparkContext
from pyspark.sql import SparkSession
#!pip install neattext
import neattext.functions as nfx
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB


In [2]:
sc=SparkContext(master='local[2]')
print(sc)

23/08/16 12:55:40 WARN Utils: Your hostname, SailhOsman resolves to a loopback address: 127.0.1.1; using 172.22.57.122 instead (on interface eth0)
23/08/16 12:55:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/16 12:55:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/16 12:55:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/16 12:55:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/08/16 12:55:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


<SparkContext master=local[2] appName=pyspark-shell>


In [3]:
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [4]:
spark=SparkSession.builder.appName("Text Mining").getOrCreate()
print(spark)

In [5]:
df=pd.read_csv("udemy_courses.csv" )
print(df.head())

   course_id                                       course_title  \
0    1070968                 Ultimate Investment Banking Course   
1    1113822  Complete GST Course & Certification - Grow You...   
2    1006314  Financial Modeling for Business Analysts and C...   
3    1210588  Beginner to Pro - Financial Analysis in Excel ...   
4    1011058       How To Maximize Your Profits Trading Options   

                                                 url  is_paid  price  \
0  https://www.udemy.com/ultimate-investment-bank...     True    200   
1      https://www.udemy.com/goods-and-services-tax/     True     75   
2  https://www.udemy.com/financial-modeling-for-b...     True     45   
3  https://www.udemy.com/complete-excel-finance-c...     True     95   
4  https://www.udemy.com/how-to-maximize-your-pro...     True    200   

   num_subscribers  num_reviews  num_lectures               level  \
0             2147           23            51          All Levels   
1             2792        

In [6]:
df.shape
df.columns

Index(['course_id', 'course_title', 'url', 'is_paid', 'price',
       'num_subscribers', 'num_reviews', 'num_lectures', 'level',
       'content_duration', 'published_timestamp', 'subject'],
      dtype='object')

In [8]:
df2=df[['course_title', 'subject']]
df2.shape
print(df2.head())

                                        course_title           subject
0                 Ultimate Investment Banking Course  Business Finance
1  Complete GST Course & Certification - Grow You...  Business Finance
2  Financial Modeling for Business Analysts and C...  Business Finance
3  Beginner to Pro - Financial Analysis in Excel ...  Business Finance
4       How To Maximize Your Profits Trading Options  Business Finance


In [9]:
## Text pre-processing- Remove noise- lower case, remove punct, remove stopwords
########## normalize, then Lammatize and Stemming @@@@@@@@@@@@@@@@@@@@@
dir(nfx)

['BTC_ADDRESS_REGEX',
 'CURRENCY_REGEX',
 'CURRENCY_SYMB_REGEX',
 'Counter',
 'DATE_REGEX',
 'EMAIL_REGEX',
 'EMOJI_REGEX',
 'HASTAG_REGEX',
 'MASTERCard_REGEX',
 'MD5_SHA_REGEX',
 'MOST_COMMON_PUNCT_REGEX',
 'NUMBERS_REGEX',
 'PHONE_REGEX',
 'PoBOX_REGEX',
 'SPECIAL_CHARACTERS_REGEX',
 'STOPWORDS',
 'STOPWORDS_de',
 'STOPWORDS_en',
 'STOPWORDS_es',
 'STOPWORDS_fr',
 'STOPWORDS_ru',
 'STOPWORDS_yo',
 'STREET_ADDRESS_REGEX',
 'TextFrame',
 'URL_PATTERN',
 'USER_HANDLES_REGEX',
 'VISACard_REGEX',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__generate_text',
 '__loader__',
 '__name__',
 '__numbers_dict',
 '__package__',
 '__spec__',
 '_lex_richness_herdan',
 '_lex_richness_maas_ttr',
 'clean_text',
 'defaultdict',
 'digit2words',
 'extract_btc_address',
 'extract_currencies',
 'extract_currency_symbols',
 'extract_dates',
 'extract_emails',
 'extract_emojis',
 'extract_hashtags',
 'extract_html_tags',
 'extract_mastercard_addr',
 'extract_md5sha',
 'extract_numbers',
 'extr

In [12]:
df['course_title_clean']=df['course_title'].apply(nfx.remove_stopwords)
df['course_title_clean']=df['course_title_clean'].apply(nfx.remove_special_characters)
df['course_title_clean']=df['course_title_clean'].str.lower()


0                   Ultimate Investment Banking Course
1    Complete GST Course & Certification - Grow You...
2    Financial Modeling for Business Analysts and C...
3    Beginner to Pro - Financial Analysis in Excel ...
4         How To Maximize Your Profits Trading Options
Name: course_title, dtype: object 0                   ultimate investment banking course
1    complete gst course  certification  grow practice
2     financial modeling business analysts consultants
3          beginner pro  financial analysis excel 2017
4                     maximize profits trading options
Name: course_title_clean, dtype: object


In [13]:
print(df['course_title'].head(), df['course_title_clean'].head())

0                   Ultimate Investment Banking Course
1    Complete GST Course & Certification - Grow You...
2    Financial Modeling for Business Analysts and C...
3    Beginner to Pro - Financial Analysis in Excel ...
4         How To Maximize Your Profits Trading Options
Name: course_title, dtype: object 0                   ultimate investment banking course
1    complete gst course  certification  grow practice
2     financial modeling business analysts consultants
3          beginner pro  financial analysis excel 2017
4                     maximize profits trading options
Name: course_title_clean, dtype: object
